## Python で始める機器分析データの解析とケモメトリックス  
### 8.3. 線形重回帰 (MLR)

In [ ]:
import numpy
import pandas
from matplotlib import pyplot

In [ ]:
data = pandas.read_csv("data3.csv", header=0, index_col=0).T
data.index = pandas.read_csv("prop3.csv", header=0, index_col=0).iloc[:, 2].values
data = ((data.T - data.T.mean()) / data.T.std()).T
from scipy.signal import savgol_filter
buff = data
buff = savgol_filter(buff, 7, 2, 0)
buff = savgol_filter(buff, 7, 2, 1)
buff = savgol_filter(buff, 7, 2, 0)
buff = savgol_filter(buff, 7, 2, 1)
data = pandas.DataFrame(buff, index=data.index, columns=data.columns)
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, train_size=0.6, random_state=8)

In [ ]:
#p.138
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(train.values, train.index)

In [ ]:
#p.138
x = data.columns
y = model.coef_
pyplot.plot(x, y)
pyplot.show()
print("b0 =", model.intercept_)

In [ ]:
spec = train.iloc[0]
print(train.index[0])
print(model.predict([spec.values])[0])
print(spec.values @ model.coef_ + model.intercept_)

In [ ]:
#p.139
calibration = model.predict(train.values)
pyplot.figure(figsize=(4, 4))
pyplot.scatter(train.index, calibration)
pyplot.plot([data.index.min(), data.index.max()], [data.index.min(), data.index.max()])
pyplot.show()
from sklearn.metrics import mean_squared_error, r2_score
print("RMSEC =", numpy.sqrt(mean_squared_error(train.index, calibration)))
print("R^2 =", r2_score(train.index, calibration))

In [ ]:
#p.140
from scipy.signal import find_peaks
peakindex = find_peaks(train.var(), prominence=1.0e-8)[0]
peak = train.var().iloc[peakindex]
train.var().plot()
pyplot.scatter(peak.index, peak.values)
pyplot.show()
print(peak.index.tolist())

In [ ]:
#p.141
train_peaks = train.iloc[:, peakindex]
test_peaks = test.iloc[:, peakindex]
model = LinearRegression().fit(train_peaks.values, train_peaks.index)
calibration = model.predict(train_peaks.values)
validation = model.predict(test_peaks.values)

In [ ]:
#p.142
pyplot.figure(figsize=(4, 4))
pyplot.plot([data.index.min(), data.index.max()], [data.index.min(), data.index.max()], c="black")
pyplot.scatter(train.index, calibration, label="calibration")
pyplot.scatter(test.index, validation, label="validation")
pyplot.legend()
pyplot.show()
from sklearn.metrics import mean_squared_error, r2_score
print("RMSEC =", numpy.sqrt(mean_squared_error(train.index, calibration)))
print("R^2 =", r2_score(train.index, calibration))
print("")
print("RMSEV =", numpy.sqrt(mean_squared_error(test.index, validation)))
print("R^2 =", r2_score(test.index, validation))

(c) Shigeaki Morita